In [4]:
import os
from dotenv import load_dotenv
from mongoengine import connect, disconnect
import pandas as pd
from pycoshark.mongomodels import Issue, Project, Commit
import re

In [5]:
# Load environment variables from .env file
load_dotenv('.env')

# Retrieve the password from the environment variable
user = os.getenv('DB_USER')
password = os.getenv('DB_PW')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

disconnect()

connect(host=f'mongodb://{user}:{password}@{db_host}:{db_port}/{db_name}?authSource=admin')


MongoClient(host=['37.114.42.132:9001'], document_class=dict, tz_aware=False, connect=True, authsource='admin', read_preference=Primary())

In [6]:
Project.objects.get(name="nova")

<Project: Project object>

In [7]:
# nova_ids = [1294939,
#             1300788,
#             1307791,
#             1314677,
#             1336127,
#             1343080,
#             1367363,
#             1370177,
#             1370590,
#             1371677,
#             1375379,
#             1381468,
#             1392798,
#             1402535,
#             1402728,
#             1403544,
#             1419002,
#             1424647,
#             1434855,
#             1437855,
#             1438331,
#             1441000,
#             1443697,
#             1443970,
#             1447249,
#             1453274,
#             1466390,
#             1467451,
#             1475356,
#             1475911,
#             1481078,
#             1481164,
#             1486541,
#             1505471]

# # nova_bfc = [
# #     "e08ce4de920b68c84ac45be8a657a95113688780",
# #     "98fb5e56ef96c0c6a8754276e5f8c9c0b7e50fed",
# #     "54f4600c82241ad6ae6768d3dcd1e1755dac4ddc",
# #     "695191fa89387d96e60120ff32965493c844e7f5",
# #     "6ddd9f93f82427ce909c7773f7a806361035a0b2",
# #     "e516de74c192591e890f4e9586d8e70bb8d9203e",
# #     "5f93c841a80663b68da2fb04df78d5acd0754d68",
# #     "a7de013891da5afd43c2aa2dd6dad61048799230",
# #     "de62c0d8361bf9dae16a8ff1672229fee1c15e5e",
# #     "833357301bc80a27422f7bf081fae2d3da730a24",
# #     "4f0547d4978172e29eb328bceb404335da1b9e0a",
# #     "fdcf358eaeef6edb5c8d2dcc94f906a22882544a",
# #     "2aea3a3d54cbe0dc9ce2b8c504818baeb1542677",
# #     "2833f8c08fcfb7961b3c64b285ceff958bf5a05e",
# #     "1b2aa11758fd52acdec0289777d276303555903b",
# #     "2ad3009f3595e701a866f265263ca3a0a8ef09dc",
# #     "b838ca28e0ffda21b298a4514a647fc74c154f4a",
# #     "8f060f07c7eeb1d1356f0ce6c0e1ca6ec4ec0b96",
# #     "ae3c63502096b9e16bd02f356286e396e4abb0de",
# #     "c5dfd2c3ef773b29666fae3fe75bf7548044dbf5",
# #     "a37bc78ed57aeabbb87b26f77fd785db3ee6a9ba",
# #     "3946cfd900b18917fccdc7330bc4f070d7abd81d",
# #     "3a77c7c570acee40100f2fae77ef5b461997b78e",
# #     "f697befdd3a0f9b81ff9a74f55e4460cd3783692",
# #     "7471b5a0924c8cb90b94ea122967b422d35d9c69",
# #     "96a2283c1a07f0298c57f57d8c4112c1c33b6128",
# #     "d4d23124a54a62607f710c44a871c0810105729a",
# #     "a02deffcfab6cc9a7b0764dd36b8514f6cb1a108",
# #     "7c0e2238eb003ac0c620f4b63fa92baa6675e724",
# #     "74e140faf23d07e23c33f707d2278679fbc5bbde",
# #     "20847c25a8157a10b765387ff8dbda31f8f4e91a",
# #     "0ed3f33028b539877abf51b5a36a0f8e5c0a5927",
# #     "812d75ecb59c3198cf40ae81cfc97a99b5ff1a50",
# #     "5252bba03e43c71f90cb2a657e6a7f396d04be75"
# # ]

# nova_bic = [
#     "542958fac7445a14dea",
#     "e517e5668ab",
#     "3dff4335661",
#     "88ab936",
#     "2c62e34",
#     "e7aa4022374fb35d2131a7c633212c5d6302db3d",
#     "0fcbb7b",
#     "33e3d4c6b9",
#     "7326bac8f95",
#     "5fa74bc",
#     "e71e8c2",
#     "30871e870",
#     "a9d0313c",
#     "933365a58",
#     "2bcda65d512",
#     "6956057ac490c788",
#     "2ef03c6a0a8c5705",
#     "d2cbb9a",
#     "abc656d",
#     "1b2c121f133b1",
#     "13e2bd0",
#     "7dd6a4a",
#     "f3f1a53f",
#     "447641972539f1",
#     "1682c5b",
#     "60c90f7",
#     "501ff41",
#     "0e02db2980dc4e76ef7949ba4ec131af9fc4a44d",
#     "0eb7e35fbf",
#     "e310985",
#     "31f208423405",
#     None,
#     "f5071bd1ac00ed68102d37c8025",
#     "64e167eb6",
# ]

# issue_bfc = list(zip(nova_ids, nova_bic))
# issue_bfc

issue_bfc = [(row['issue_id'], row['non_bic_revision_hash']) for (index, row) in pd.read_csv('nova_non_bic.csv', sep='\t').iterrows()]

In [8]:
result = {}

def flatten(xss):
    return [x for xs in xss for x in xs]

def get_data(commits: [Commit]):
    return [{
        "revision_hash": commit.revision_hash,
        "author_date": commit.author_date,
        "is_merge": len(commit.parents) > 1,
        "is_rebase": any([parent.author_date > commit.author_date for parent in Commit.objects(revision_hash__in=commit.parents)]),
    } for commit in commits]

for issue_id, rev_hash in issue_bfc:    

    if not rev_hash:
        print(issue_id, "No root found")
        continue

    regex = re.compile(rev_hash)
    root = Commit.objects(revision_hash=regex).first()


    parents = Commit.objects(revision_hash__in=root.parents)
    grandparents = Commit.objects(revision_hash__in=flatten([parent.parents for parent in parents]))
    children =Commit.objects(parents=root.revision_hash)
    children_parents = Commit.objects(revision_hash__in=flatten([child.parents for child in children]))

    result[f'{issue_id}'] = {
        'root': rev_hash,
        'true_root': get_data([root])[0],
        'parents': get_data(parents),
        'grandparents': get_data(grandparents),
        'children': get_data(children),
        'children_parents': get_data(children_parents)
    }

result = pd.DataFrame(result).T

In [9]:
result = result.explode('parents')
result = result.explode('grandparents')
result = result.explode('children')
result = result.explode('children_parents')

In [10]:
result

,root,true_root,parents,grandparents,children,children_parents
NOVA 1294939,542958fac7445a14deaae146b0d4047aa90a2fab,{'revision_hash': '542958fac7445a14deaae146b0d...,{'revision_hash': '79121d9a11d10b80fd3eee53073...,{'revision_hash': '8bc9586056fe2f455a23b342d3d...,{'revision_hash': '311652325b93f6e2675bdcc52c4...,{'revision_hash': '542958fac7445a14deaae146b0d...
NOVA 1294939,542958fac7445a14deaae146b0d4047aa90a2fab,{'revision_hash': '542958fac7445a14deaae146b0d...,{'revision_hash': '79121d9a11d10b80fd3eee53073...,{'revision_hash': '8bc9586056fe2f455a23b342d3d...,{'revision_hash': '311652325b93f6e2675bdcc52c4...,{'revision_hash': 'b66a092a7a9ab2363ce23cdfd09...
NOVA 1294939,542958fac7445a14deaae146b0d4047aa90a2fab,{'revision_hash': '542958fac7445a14deaae146b0d...,{'revision_hash': '79121d9a11d10b80fd3eee53073...,{'revision_hash': '8bc9586056fe2f455a23b342d3d...,{'revision_hash': '9776175fc3b2331168651176ec5...,{'revision_hash': '542958fac7445a14deaae146b0d...
NOVA 1294939,542958fac7445a14deaae146b0d4047aa90a2fab,{'revision_hash': '542958fac7445a14deaae146b0d...,{'revision_hash': '79121d9a11d10b80fd3eee53073...,{'revision_hash': '8bc9586056fe2f455a23b342d3d...,{'revision_hash': '9776175fc3b2331168651176ec5...,{'revision_hash': 'b66a092a7a9ab2363ce23cdfd09...
NOVA 1300788,e517e5668ab20214432e06ef87f7b16776edaf7c,{'revision_hash': 'e517e5668ab20214432e06ef87f...,{'revision_hash': '3f2b22499d9e94ae3ac80786752...,{'revision_hash': 'a9aa1a402b64751614fbc98e149...,{'revision_hash': '8bac24e5b7252b3f23c5473cdf2...,{'revision_hash': '049bfbcc29d1a5ff45446d88a06...
...,...,...,...,...,...,...
NOVA 1486541,f5071bd1ac00ed68102d37c8025d36df6777cd9e,{'revision_hash': 'f5071bd1ac00ed68102d37c8025...,{'revision_hash': '4897b18a406aa1fa3ba4916f9d9...,{'revision_hash': '784372985907dd88771e4ee1ff3...,{'revision_hash': '87f35cfcc7d26b89dcc084be337...,{'revision_hash': 'f5071bd1ac00ed68102d37c8025...
NOVA 1505471,64e167eb62bd3483b2947ec8de218453c116bd93,{'revision_hash': '64e167eb62bd3483b2947ec8de2...,{'revision_hash': 'ab77c4e1b8e4c500a9372e290e6...,{'revision_hash': '717c9755805aa06340130b6bc4e...,{'revision_hash': 'd751a58d0a909a5ab9605a4fc70...,{'revision_hash': '0e4e6edb06951dc836651697c2d...
NOVA 1505471,64e167eb62bd3483b2947ec8de218453c116bd93,{'revision_hash': '64e167eb62bd3483b2947ec8de2...,{'revision_hash': 'ab77c4e1b8e4c500a9372e290e6...,{'revision_hash': '717c9755805aa06340130b6bc4e...,{'revision_hash': 'd751a58d0a909a5ab9605a4fc70...,{'revision_hash': '64e167eb62bd3483b2947ec8de2...
NOVA 1505471,64e167eb62bd3483b2947ec8de218453c116bd93,{'revision_hash': '64e167eb62bd3483b2947ec8de2...,{'revision_hash': 'ab77c4e1b8e4c500a9372e290e6...,{'revision_hash': '9adc3a456a4a38fd9e372ddd7ee...,{'revision_hash': 'd751a58d0a909a5ab9605a4fc70...,{'revision_hash': '0e4e6edb06951dc836651697c2d...


In [11]:
result[['true_root_revision_hash', 'true_root_author_date', 'true_root_is_merge', 'true_root_is_rebase']] = result['true_root'].apply(pd.Series)
result.drop('true_root', axis=1, inplace=True)

result[['parents_revision_hash', 'parents_author_date', 'parents_is_merge', 'parents_is_rebase']] = result['parents'].apply(pd.Series)
result.drop('parents', axis=1, inplace=True)

result[['grandparents_revision_hash', 'grandparents_author_date', 'grandparents_is_merge', 'grandparents_is_rebase']] = result['grandparents'].apply(pd.Series)
result.drop('grandparents', axis=1, inplace=True)

result[['children_revision_hash', 'children_author_date', 'children_is_merge', 'children_is_rebase']] = result['children'].apply(pd.Series)
result.drop('children', axis=1, inplace=True)

result[['children_parents_revision_hash', 'children_parents_author_date', 'children_parents_is_merge', 'children_parents_is_rebase']] = result['children_parents'].apply(pd.Series)
result.drop('children_parents', axis=1, inplace=True)


In [12]:
result

,root,true_root_revision_hash,true_root_author_date,true_root_is_merge,true_root_is_rebase,parents_revision_hash,parents_author_date,parents_is_merge,parents_is_rebase,grandparents_revision_hash,...,grandparents_is_merge,grandparents_is_rebase,children_revision_hash,children_author_date,children_is_merge,children_is_rebase,children_parents_revision_hash,children_parents_author_date,children_parents_is_merge,children_parents_is_rebase
NOVA 1294939,542958fac7445a14deaae146b0d4047aa90a2fab,542958fac7445a14deaae146b0d4047aa90a2fab,2014-01-16 22:36:32,False,False,79121d9a11d10b80fd3eee53073913e0f66f0949,2014-01-16 20:59:06,False,True,8bc9586056fe2f455a23b342d3da5473e9d3cbfc,...,True,False,311652325b93f6e2675bdcc52c416bd06f12fd3b,2014-02-14 19:35:15,True,False,542958fac7445a14deaae146b0d4047aa90a2fab,2014-01-16 22:36:32,False,False
NOVA 1294939,542958fac7445a14deaae146b0d4047aa90a2fab,542958fac7445a14deaae146b0d4047aa90a2fab,2014-01-16 22:36:32,False,False,79121d9a11d10b80fd3eee53073913e0f66f0949,2014-01-16 20:59:06,False,True,8bc9586056fe2f455a23b342d3da5473e9d3cbfc,...,True,False,311652325b93f6e2675bdcc52c416bd06f12fd3b,2014-02-14 19:35:15,True,False,b66a092a7a9ab2363ce23cdfd09e8f564c986c06,2014-02-14 18:40:28,True,False
NOVA 1294939,542958fac7445a14deaae146b0d4047aa90a2fab,542958fac7445a14deaae146b0d4047aa90a2fab,2014-01-16 22:36:32,False,False,79121d9a11d10b80fd3eee53073913e0f66f0949,2014-01-16 20:59:06,False,True,8bc9586056fe2f455a23b342d3da5473e9d3cbfc,...,True,False,9776175fc3b2331168651176ec56f9adc5d463dd,2014-01-17 01:03:41,False,False,542958fac7445a14deaae146b0d4047aa90a2fab,2014-01-16 22:36:32,False,False
NOVA 1294939,542958fac7445a14deaae146b0d4047aa90a2fab,542958fac7445a14deaae146b0d4047aa90a2fab,2014-01-16 22:36:32,False,False,79121d9a11d10b80fd3eee53073913e0f66f0949,2014-01-16 20:59:06,False,True,8bc9586056fe2f455a23b342d3da5473e9d3cbfc,...,True,False,9776175fc3b2331168651176ec56f9adc5d463dd,2014-01-17 01:03:41,False,False,b66a092a7a9ab2363ce23cdfd09e8f564c986c06,2014-02-14 18:40:28,True,False
NOVA 1300788,e517e5668ab20214432e06ef87f7b16776edaf7c,e517e5668ab20214432e06ef87f7b16776edaf7c,2013-06-05 07:16:16,False,True,3f2b22499d9e94ae3ac80786752223f43d24492b,2013-06-13 01:30:01,True,False,a9aa1a402b64751614fbc98e14973d867a1bfca7,...,False,True,8bac24e5b7252b3f23c5473cdf2e17a9740bdb98,2013-06-14 19:11:04,True,False,049bfbcc29d1a5ff45446d88a068990600505bed,2013-06-14 19:08:29,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NOVA 1486541,f5071bd1ac00ed68102d37c8025d36df6777cd9e,f5071bd1ac00ed68102d37c8025d36df6777cd9e,2014-01-31 14:20:47,False,True,4897b18a406aa1fa3ba4916f9d9cfde2b064f406,2014-03-06 01:55:17,True,False,784372985907dd88771e4ee1ff32305022b49e78,...,False,False,87f35cfcc7d26b89dcc084be337d97a213079e77,2014-02-04 11:02:45,False,False,f5071bd1ac00ed68102d37c8025d36df6777cd9e,2014-01-31 14:20:47,False,True
NOVA 1505471,64e167eb62bd3483b2947ec8de218453c116bd93,64e167eb62bd3483b2947ec8de218453c116bd93,2012-08-06 16:37:12,False,True,ab77c4e1b8e4c500a9372e290e658952a2441627,2012-11-26 18:47:22,True,False,717c9755805aa06340130b6bc4e7de8f81ea8e57,...,False,True,d751a58d0a909a5ab9605a4fc707c1027d710755,2012-11-26 23:57:25,True,False,0e4e6edb06951dc836651697c2dc6bd0a7d0f862,2012-11-26 21:43:09,True,False
NOVA 1505471,64e167eb62bd3483b2947ec8de218453c116bd93,64e167eb62bd3483b2947ec8de218453c116bd93,2012-08-06 16:37:12,False,True,ab77c4e1b8e4c500a9372e290e658952a2441627,2012-11-26 18:47:22,True,False,717c9755805aa06340130b6bc4e7de8f81ea8e57,...,False,True,d751a58d0a909a5ab9605a4fc707c1027d710755,2012-11-26 23:57:25,True,False,64e167eb62bd3483b2947ec8de218453c116bd93,2012-08-06 16:37:12,False,True
NOVA 1505471,64e167eb62bd3483b2947ec8de218453c116bd93,64e167eb62bd3483b2947ec8de218453c116bd93,2012-08-06 16:37:12,False,True,ab77c4e1b8e4c500a9372e290e658952a2441627,2012-11-26 18:47:22,True,False,9adc3a456a4a38fd9e372ddd7ee2fc6abeddbb0d,...,True,False,d751a58d0a909a5ab9605a4fc7

In [13]:
u = set()

prefix = ""

def my_function(row):
    if row[f'{prefix}_revision_hash'] in u:
        row[f'{prefix}_revision_hash'] = None
        row[f'{prefix}_author_date'] = None
        row[f'{prefix}_is_merge'] = None
        row[f'{prefix}_is_rebase'] = None
    else:
        u.add(row[f'{prefix}_revision_hash'])

    return row

for prefix in ['true_root', 'parents', 'grandparents', 'children', 'children_parents']:
    u = set()
    result = result.apply(my_function, axis=1)


In [14]:
result

,root,true_root_revision_hash,true_root_author_date,true_root_is_merge,true_root_is_rebase,parents_revision_hash,parents_author_date,parents_is_merge,parents_is_rebase,grandparents_revision_hash,...,grandparents_is_merge,grandparents_is_rebase,children_revision_hash,children_author_date,children_is_merge,children_is_rebase,children_parents_revision_hash,children_parents_author_date,children_parents_is_merge,children_parents_is_rebase
NOVA 1294939,542958fac7445a14deaae146b0d4047aa90a2fab,542958fac7445a14deaae146b0d4047aa90a2fab,2014-01-16 22:36:32,False,False,79121d9a11d10b80fd3eee53073913e0f66f0949,2014-01-16 20:59:06,False,True,8bc9586056fe2f455a23b342d3da5473e9d3cbfc,...,True,False,311652325b93f6e2675bdcc52c416bd06f12fd3b,2014-02-14 19:35:15,True,False,542958fac7445a14deaae146b0d4047aa90a2fab,2014-01-16 22:36:32,False,False
NOVA 1294939,542958fac7445a14deaae146b0d4047aa90a2fab,None,NaT,None,None,None,NaT,None,None,None,...,None,None,None,NaT,None,None,b66a092a7a9ab2363ce23cdfd09e8f564c986c06,2014-02-14 18:40:28,True,False
NOVA 1294939,542958fac7445a14deaae146b0d4047aa90a2fab,None,NaT,None,None,None,NaT,None,None,None,...,None,None,9776175fc3b2331168651176ec56f9adc5d463dd,2014-01-17 01:03:41,False,False,None,NaT,None,None
NOVA 1294939,542958fac7445a14deaae146b0d4047aa90a2fab,None,NaT,None,None,None,NaT,None,None,None,...,None,None,None,NaT,None,None,None,NaT,None,None
NOVA 1300788,e517e5668ab20214432e06ef87f7b16776edaf7c,e517e5668ab20214432e06ef87f7b16776edaf7c,2013-06-05 07:16:16,False,True,3f2b22499d9e94ae3ac80786752223f43d24492b,2013-06-13 01:30:01,True,False,a9aa1a402b64751614fbc98e14973d867a1bfca7,...,False,True,8bac24e5b7252b3f23c5473cdf2e17a9740bdb98,2013-06-14 19:11:04,True,False,049bfbcc29d1a5ff45446d88a068990600505bed,2013-06-14 19:08:29,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NOVA 1486541,f5071bd1ac00ed68102d37c8025d36df6777cd9e,None,NaT,None,None,None,NaT,None,None,784372985907dd88771e4ee1ff32305022b49e78,...,False,False,None,NaT,None,None,None,NaT,None,None
NOVA 1505471,64e167eb62bd3483b2947ec8de218453c116bd93,64e167eb62bd3483b2947ec8de218453c116bd93,2012-08-06 16:37:12,False,True,ab77c4e1b8e4c500a9372e290e658952a2441627,2012-11-26 18:47:22,True,False,717c9755805aa06340130b6bc4e7de8f81ea8e57,...,False,True,d751a58d0a909a5ab9605a4fc707c1027d710755,2012-11-26 23:57:25,True,False,0e4e6edb06951dc836651697c2dc6bd0a7d0f862,2012-11-26 21:43:09,True,False
NOVA 1505471,64e167eb62bd3483b2947ec8de218453c116bd93,None,NaT,None,None,None,NaT,None,None,None,...,None,None,None,NaT,None,None,64e167eb62bd3483b2947ec8de218453c116bd93,2012-08-06 16:37:12,False,True
NOVA 1505471,64e167eb62bd3483b2947ec8de218453c116bd93,None,NaT,None,None,None,NaT,None,None,9adc3a456a4a38fd9e372ddd7ee2fc6abeddbb0d,...,True,False,None,NaT,None,None,None,NaT,None,None


In [15]:
result.to_csv('nova_non_bic_family.csv')